# Covariance Estimation Demo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from geospatial_neural_adapter.cpp_extensions import spatial_utils

In [ ]:
print("Available functions in spatial_utils:")
for attr in dir(spatial_utils):
    if not attr.startswith('_'):
        print(f"  - {attr}")

### Estimate Covariance

In [ ]:
# Example for estimate_covariance function
print("Demonstrating estimate_covariance...")

# Create repeated observations at the same locations
n_obs = 10  # number of repeated observations
n_locations = 50  # number of locations
np.random.seed(42)
n_points = 50

# Create random spatial locations
x = np.random.uniform(0, 10, n_points)
y = np.random.uniform(0, 10, n_points)
locations = np.column_stack([x, y])

# Create n x p matrix: n observations x p locations
Y = np.zeros((n_obs, n_locations))

# Generate repeated observations at each location
for i in range(n_obs):
    # Add some spatial correlation and noise
    spatial_trend = 2 * np.sin(locations[:, 0]/2) + 1.5 * np.cos(locations[:, 1]/3)
    noise = np.random.normal(0, 0.3, n_locations)
    Y[i, :] = spatial_trend + noise

print(f"Y shape: {Y.shape}")  # Should be (10, 50)

# Create basis functions (phi) for the locations
# phi should be p x k where p = number of locations, k = number of basis functions
n_basis = 7
phi = np.zeros((n_locations, n_basis))

# Create spatial basis functions
for j in range(n_basis):
    if j < 2:
        phi[:, j] = locations[:, j]  # x, y coordinates
    elif j < 4:
        phi[:, j] = locations[:, j-2]**2  # x^2, y^2
    elif j < 6:
        phi[:, j] = np.sin(locations[:, j-4])  # sin(x), sin(y)
    else:
        phi[:, j] = np.cos(locations[:, j-6])  # cos(x)

print(f"phi shape: {phi.shape}")  # Should be (50, 7)

result = spatial_utils.estimate_covariance(phi, Y)
print("C++ returned a dictionary with keys:")
for key, value in result.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: shape {value.shape}")
    else:
        print(f"  {key}: {type(value)}")

covariance_matrix = result['estimated_covariance']
print(f"C++ Covariance matrix shape: {covariance_matrix.shape}")
print(f"C++ Covariance range: [{covariance_matrix.min():.3f}, {covariance_matrix.max():.3f}]")
